In [ ]:
import glob
import random
import os
import sys

import h5py
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import pandas as pd
from scipy.stats import zscore,kstest
import seaborn as sns

#from placecode.expinfo import MultipleFilesFoundError

#import my functions
import sys
sys.path.append('..')
from placecode.spatial_coding_functions import make_firing_rate_maps,make_binary,adding_parameters,cell_morphology,TunedVector,KstestPlaceCells

In [ ]:
#initializing the folder with the data

home_folder=f"{exported_data_folder}/{mouse_ID}/{mouse_ID}_{condition}"

data_folder=f'{home_folder}' #r means that I will treat the string as a raw string (/ are special characters)
results_folder=f'{home_folder}'

# Check if results folder exists, create it if not
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

cnmf_file=glob.glob(f"{home_folder}/*.hdf5") #data from cnmf
belt_file=glob.glob(f"{home_folder}/*.h5") #data from lab view and lfp

#check that there is only one file out of each

if len(cnmf_file)>1 or len(belt_file)>1:
    raise MultipleFilesFoundError("error.two files with similar conditions found")
else:
    print ("all good with naming the files. continue")

In [ ]:
#since glob glob is a list I take the first element out of belt file and cmf file. I have checked begore that it is only one


cnmf_file,belt_file=cnmf_file[0],belt_file[0]

#preprocessing the data

#opening the hpf5 file
fluo_hdf=h5py.File(cnmf_file)['estimates']['C']

#convert it in a dataframe
fluo_hdf=pd.DataFrame(fluo_hdf)


#opening the stripe folder
stripe_hdf=h5py.File(belt_file)['inferred']['belt_dict']['stripes']
stripe_hdf=pd.DataFrame(stripe_hdf)
fluo_hdf_r=fluo_hdf.T #swap columns and rows in fluo hdf
n_cells=fluo_hdf_r.shape[1]

#normazing the whole panda frame by applying z score as well as keeping the raw data for visualization
raw_data=fluo_hdf_r
fluo_hdf_r=fluo_hdf_r.apply(zscore,axis=0)

#identifying and storing the number of units
units_n=fluo_hdf_r.shape[1]



1.creating panda frames for running distance, speed, time and immobility
2.adding all of them into the intial panda frame so that I can analyze everything at the same time

In [ ]:
fluo_z_score,fluo_raw_data= adding_parameters(fluo_hdf_r,raw_data,belt_file)


I want to plot as I test the different distances every time in every round to compare.
all the rounds should be approximately 1500

In [ ]:
wanted_distance=belt_length*10
n_rounds=int(fluo_z_score['Rounds'].max())
rounds=[]
for round in range(1,n_rounds+1):
    dis=fluo_z_score[fluo_z_score['Rounds']==round].iloc[-1]['Distance']
    if abs(dis-wanted_distance)<15:
        rounds.append(round)

num_rounds=len(rounds)
print(f'Rounds: {rounds}\n Number of rounds: {num_rounds}')




In [ ]:

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

with h5py.File(f'{save_folder}/output_info.h5', 'w') as file:
    file.create_dataset('rounds', data=rounds)

processing the data

In [ ]:
data=fluo_z_score                    #renaming data so that it is easier to process them
data=data[data['Speed']>0]   #taking only the timepoints when the mouse moves
data=data[data['Rounds'].isin(rounds)]         #only taking the data of 8 rounds because this is where the belt approximates 150 cm

calculate spatial firing map for each round indivudually. then I can average them easily.
no need to do it from the beginning

In [ ]:
# Extract relevant columns
fluorescence_traces = data.iloc[:, :units_n]  # Columns containing fluorescence traces
distance = data['Distance']  # Column containing distance on the linear belt
speed = data['Speed']   # Column containing speed
running = data['Running'] # Column containing running status


num_units=units_n

#making the firing rate map for every cell and every round for both zscore and raw data

firing_rate_maps=make_firing_rate_maps(fluo_z_score,rounds,num_units,num_bins)
firing_rate_maps_raw_data=make_firing_rate_maps(fluo_raw_data,rounds,num_units,num_bins)

#calculate the average firing rate map for every cell

avr_firing_rate_maps=np.mean(firing_rate_maps,axis=1)
avr_firing_rate_maps_raw_data=np.mean(firing_rate_maps_raw_data,axis=1)

#caclulate the event map for every cell
#meaning I will make a binary array where every peak of the activity is 1 and everything else is zero

events_per_cell=make_binary(firing_rate_maps,peak_threshold=3,peak_distance=10)




In [ ]:
#storing the data

with h5py.File(f'{save_folder}/output_info.h5', 'a') as file:
    file.create_dataset('number of units',data=units_n)
    file.create_dataset('firing_rate_maps', data=firing_rate_maps)
    file.create_dataset('events_per_cell', data=events_per_cell)
    file.create_dataset('avr_firing_rate_maps', data=avr_firing_rate_maps)

In [ ]:
#identify the units which have at least a certain amount of events

cells_with_events=[]
silent_cells=[]
for i in range(units_n):
    if np.sum(events_per_cell[i])>n_events_threshold:
        cells_with_events.append(i)
    else:
        silent_cells.append(i)

with h5py.File(f'{save_folder}/output_info.h5', 'a') as file:
    file.create_dataset('cells_with_events', data=cells_with_events)
    file.create_dataset('silent_cells',data=silent_cells)

calculating kolmogorov smirnon analysis data for every cell so I do not do it inside the plot

In [ ]:
#extracting morphology of all cells

spatial=cell_morphology(cnmf_file)

plotting
1. z score activity per round per cell
2. average z-score activity for all the rounds
3. spatial vector analysis schematic and histogram with comparison of shuffling data
4. statistical significanc e analysis using colmogorov-smirnov test

Code for plotting everything

In [ ]:
fig_folder=f'{save_folder}'
if not os.path.exists(fig_folder):
    os.makedirs(fig_folder)


#creating pdfs for significant and non significant cells sop that I can review them after
# Create two pdf files
significant_cells = PdfPages(f'{fig_folder}/{mouse_ID}_{condition}_significant_place_cells.pdf')
non_significant_cells = PdfPages(f'{fig_folder}/{mouse_ID}_{condition}_non_significant_place_cells.pdf')


angles = np.linspace(0, 2*np.pi, num_bins, endpoint=False) #initializing the circle with 150 bins corresponding to the distance on the belt

place_cells_both_methods=[]
place_cells_tuned_vector=[]
place_cells_ks_test=[]

non_place_cells_tuned_vector=[]

for cell in cells_with_events:
    
    data=firing_rate_maps[cell]
    data_avg=avr_firing_rate_maps[cell]

    events_data = events_per_cell[cell]  # Assuming you have your actual events data
    peak_n=np.sum(events_data) #finding the number of peaks during the whole time the mouse was running

    ####################################################
    # creating the plot where I will put all of the suplots inside
    ###################################################

    fig, ax = plt.subplots(3, 2, figsize=(15, 15))
    fig.suptitle(f'Cell {cell} \n shuffles {shuffling_times}')

    ###################################################
    #plotting fluorescence for every round in the first subplot
    ##################################################

    sns.heatmap(data,ax=ax[0,0],cbar_kws={'label': 'z-score fluorescence'})
    ax[0,0].set_ylabel('Rounds')
    ax[0,0].set_title("Activity per round")
    ax[0,0].set_xlabel('Position belt (cm)')
    ax1_labels=range(1,num_rounds+1)
    ax[0,0].set_yticklabels(ax1_labels)
    custom_ticks = [0, 50, 100, 150]  # Specify the positions where you want the ticks
    custom_labels = ['0', '50', '100', '150']  # Specify the labels for the ticks
    ax[0,0].set_xticks(custom_ticks)
    ax[0,0].set_xticklabels(custom_labels)

    ###################################################
    #plotting average z=score for every round in the second subplot
    ##################################################


    avg_activity = pd.DataFrame(data_avg, columns=['']) #taking the average activity of the cell and converting it to data frame to plot it
    sns.heatmap(avg_activity.transpose(),ax=ax[0,1],cbar_kws={'label': 'z-score fluorescence'})

    #changing the size of the second subplot
    #position = ax[0,1].get_position()  # Get the current position of the subplot
    ax[0,1].set_title("Average activity")
    #position.y0 -= -0.1  # Decrease the bottom boundary
    #position.y1 *= 0.9  # Decrease the height
    #ax[0,1].set_position(position)  # Set the new position
    ax[0,1].set_ylabel('')
    ax[0,1].set_xlabel('Position belt (cm)')
    ax[0,1].set_xticks(custom_ticks)
    ax[0,1].set_xticklabels(custom_labels)

    ###################################################
    #plotting spatial tuned vector figure
    ##################################################


    # Create a polar plot with circles representing each round
    
    ax[1,0] = plt.subplot(2,2,3,projection='polar')
    ax[1,0].set_theta_direction(-1)  # Set clockwise direction
    ax[1,0].set_theta_zero_location('N')  # Set zero angle at North

    # Customize the tick labels on the circles (bins) so  that I can see the distance on the belt instead of degrees
    circle_ticks = [0, 120, 240]  # Tick positions in degrees
    circle_labels = ['0/150 cm', '50 cm', '100 cm']  # Labels for the ticks
    ax[1,0].set_xticks(np.radians(circle_ticks))  # Set the tick positions in radians
    ax[1,0].set_xticklabels(circle_labels)  # Set the tick labels

    # Plot events on the circle
    for i, events_round in enumerate(events_data):
        event_angles = angles[np.where(events_round == 1)] #I found the angles where I have events
        event_radii = np.ones_like(event_angles) * (i + 1)  # Adjust the radius for each round
        ax[1,0].scatter(event_angles, event_radii, s=100, label=f'Round {i+1}', alpha=0.7)
        for angle, radius in zip(event_angles, event_radii):
            ax[1,0].plot([0, angle], [0, radius], color='black', linestyle='-', linewidth=1)  # Connect center to event dot. better for visualization

    # Calculate average direction using Cartesian coordinates
    x_coords,y_coords=TunedVector.calculate_coordinates(data=events_data,angles=angles)

    #finding sum coordinates and direction
    sum_direction,sum_magnitude=TunedVector.calculate_avr_vector(x_coords,y_coords,peak_n)

    # Plot the average tuning vector as an arrow
    ax[1,0].arrow(0, 0, sum_direction, sum_magnitude, head_width=0, head_length=0,linewidth=4, fc='red', ec='red')

    # Customize plot
    ax[1,0].set_title(f'Events and Tuning Vectors Cell {cell}')
    ax[1,0].legend(loc='upper left',bbox_to_anchor=(0.9, 1), frameon=False)
    ax[1,0].grid(True)

    # Convert average direction from radians to degrees
    sum_direction_degrees = np.degrees(sum_direction)
    # Map the sum direction to the corresponding position on the belt
    position_on_belt = (sum_direction_degrees % 360) * (belt_length / 360)

    ###########
    #plotting the histogram of the vector lengths coming from the shuffled data and compare it with the actual vector length
    ###########

    shuffled_vector_lengths=[]

    for n in range(shuffling_times):
        #shuffling every row separately
        shuffled_events_data = events_data.copy()  # Make a copy to avoid modifying original data
        for row in shuffled_events_data:
            np.random.shuffle(row)  # Shuffle the spiking events within each round independently

        # Calculate average direction and magnitude for shuffled data. same logic as before exactly
        shuffled_x_coords,shuffled_y_coords=TunedVector.calculate_coordinates(data=shuffled_events_data,angles=angles)
        shuffled_sum_direction,shuffled_sum_magnitude=TunedVector.calculate_avr_vector(shuffled_x_coords,shuffled_y_coords,peak_n)
        shuffled_vector_lengths.append(shuffled_sum_magnitude)

        # Map the shuffled average direction to the corresponding position on the belt
        shuffled_position_on_belt = (np.degrees(shuffled_sum_direction) % 360) * (belt_length / 360)


    ###########
    #making the histogram with the shuffled data and our actual vector length
    ###########

    # Calculate histogram
    counts, bins = np.histogram(shuffled_vector_lengths, bins=20)

    # Calculate probabilities
    total_samples = len(shuffled_vector_lengths)
    probabilities = counts / total_samples

    # Plot histogram with probabilities
    ax[1,1].bar(bins[:-1], probabilities, width=np.diff(bins), color='blue', alpha=0.7)
    ax[1,1].axvline(x=sum_magnitude,color='red')
    ax[1,1].set_xlabel('Vector Length')
    ax[1,1].set_ylabel('Probability')
    # Remove right and upper axes
    ax[1,1].spines['right'].set_visible(False)
    ax[1,1].spines['top'].set_visible(False)
    ax[1,1].set_title('Histogram for vector lengths')
    ax[1,1].grid(False)


    # comparing it with the actual vector lenth now)
    # Actual vector length
    actual_length = sum_magnitude  # Example value


    # Calculate the p-value
    larger_lengths_count = np.sum(np.array(shuffled_vector_lengths) > actual_length)
    total_samples = len(shuffled_vector_lengths)
    p_value_vector = larger_lengths_count / total_samples
    # Annotate the p-value on the upper side of the plot

    # Create a legend entry with the p-value
    p_value_text = f'P-value: {p_value_vector:.4f}'
    p_value_patch = mpatches.Patch(color='none', label=p_value_text)

    # Add the legend with the custom entry
    #ax[1,1].legend(handles=[p_value_patch],loc='upper right', bbox_to_anchor=(1.25, 1.05),frameon=False)

    ###################################################
    #plotting the histogram from colmogorov smirnov test
    ##################################################


    ###########################
    # I will shuffle once. and then I will compare the shufle data with my original data. the colmogorov smiron value will be the value that I will compare later
    # later I will shuffle the data 1000 times and I will compare them to the first suffling data
    ##############################

    shuffled_ks=[] #array where I will put the ks distances where I will compare the shuffled data with my first shuffling
   
    #shuffling for the baseline
    baseline_avg=KstestPlaceCells.ks_shuffling(data,num_rounds=num_rounds,shuf_distance=belt_length)
    
    baseline_ks,_=kstest(data_avg,baseline_avg)

    for n in range(1,shuffling_times):
        shuffle_avg=KstestPlaceCells.ks_shuffling(data,num_rounds=num_rounds,shuf_distance=belt_length)
        ks_shuffle,p_value=kstest(baseline_avg,shuffle_avg)
        shuffled_ks.append(ks_shuffle)

    
    # Calculate histogram
    counts, bins = np.histogram(shuffled_ks, bins=30)

    # Calculate probabilities
    total_samples = len(shuffled_ks)
    probabilities = counts / total_samples
    # Calculate the p-value
    larger_lengths_count = np.sum(np.array(shuffled_ks) > baseline_ks)
    p_value_ks = larger_lengths_count / total_samples
    # Annotate the p-value on the upper side of the plot
    # Create a legend entry with the p-value
    p_value_text = f'P-value: {p_value_ks:.4f}'
    p_value_patch = mpatches.Patch(color='none', label=p_value_text)

    # Add the legend with the custom entry
    ax[2,1].legend(handles=[p_value_patch],loc='upper right', bbox_to_anchor=(1.25, 1.05),frameon=False)

    ax[2,1].bar(bins[:-1], probabilities, width=np.diff(bins), color='blue', alpha=0.7)
    ax[2,1].axvline(x=baseline_ks,color='red')
    ax[2,1].set_xlabel('ks Length')
    ax[2,1].set_ylabel('Probability')
    # Remove right and upper axes
    ax[2,1].spines['right'].set_visible(False)
    ax[2,1].spines['top'].set_visible(False)
    ax[2,1].set_title('colmogorov smirnov test')
    ax[2,1].grid(False)

    if p_value_vector < 0.05:
        place_cells_tuned_vector.append(cell)
    elif p_value_vector>=0.05:
        non_place_cells_tuned_vector.append(cell)

    if p_value_ks < 0.05:
        place_cells_ks_test.append(cell)

        # Check the p-value and save the figure to the appropriate PDF file
    if p_value_vector < 0.05 and p_value_ks < 0.05:
        place_cells_both_methods.append(cell)
        significant_cells.savefig(fig)
    else:
        non_significant_cells.savefig(fig)

    plt.ioff() #this is so I dont have to show the plots but only store them
    plt.close()

# Close the PDF files after saving all figures
significant_cells.close()
non_significant_cells.close()

In [ ]:
with h5py.File(f'{save_folder}/output_info.h5', 'a') as file:
    file.create_dataset('place_cell_both_methods', data=place_cells_both_methods)
    file.create_dataset('place_cells_tuned_vector', data=place_cells_tuned_vector)
    file.create_dataset('place_cells_ks_test', data=place_cells_ks_test)
    file.create_dataset('non_place_cells_tuned_vector',data=non_place_cells_tuned_vector)



code for plotting everyting including calcium traces

In [ ]:
# #INITIAL PARAMETERS


# n_cells=units_n #number of cells that I want to permutate
# peak_threshold=input_info["peak_threshold"]  #amount of sd from the baseline activity
# peak_distance=input_info["peak_distance"]  #distance in cells that I am looking for local maxima
# belt_length=input_info["belt_length"] #(in cm)
# shuffling_times=input_info["shuffling_times"]  #time of shuffling for statistical comparison

# #storing round info to jason
# peak_parameters_info= {
#     "threshold":peak_threshold,
#     "peak distance (moving window for peaks)":peak_distance
# }

# mouse_info.update(peak_parameters_info)
# with open(jason_path, "w") as json_file:
#     json.dump(mouse_info, json_file,indent=jason_indent)



# #creating pdfs for significant and non significant cells sop that I can review them after
# # Create two pdf files
# # significant_cells = PdfPages(results_folder+'significant_place_cells.pdf')
# # non_significant_cells = PdfPages(results_folder+'non_significant_place_cells.pdf')


# angles = np.linspace(0, 2*np.pi, num_bins, endpoint=False) #initializing the circle with 150 bins corresponding to the distance on the belt

# place_cells_both_methods=[]
# place_cells_tuned_vector=[]
# place_cells_ks_test=[]



# for cell in cells_with_events[:5]:
    
#     data=firing_rate_maps[cell]
#     events_data = events_per_cell[cell]  # Assuming you have your actual events data
    
#     peak_n=np.sum(events_data) #finding the number of peaks during the whole time the mouse was running

#     ####################################################
#     # creating the plot where I will put all of the suplots inside
#     ###################################################

#     fig, ax = plt.subplots(5, 2, figsize=(20, 20))
#     plt.subplots_adjust(hspace=0.4,wspace=0.2)
#     fig.suptitle(f'Cell {cell} \n shuffles {shuffling_times}')

   
#     import placecode.plots as placecode_plots


#     placecode_plots.plot_fluorescence_per_round(
#         binned_rounds=firing_rate_maps_raw_data[cell], 
#         ax=ax[0,0],
#     )

#     placecode_plots.plot_average_raw_fluorescence(
#         data=avr_firing_rate_maps_raw_data[cell], 
#         ax=ax[0,1]
#     )

#     placecode_plots.plot_zscore_fluorescence_per_round(
#         data=firing_rate_maps[cell],
#         ax=ax[1,0])
    

#     data_avg=avr_firing_rate_maps[cell]
#     placecode_plots.plot_average_zscore_activity(
#         data=data_avg,
#         ax=ax[1,1])

#     ###################################################
#     #plotting spatial tuned vector figure
#     ##################################################


#     # Create a polar plot with circles representing each round
    
#     ax[2,0] = plt.subplot(2,2,3,projection='polar')
#     ax[2,0] .set_theta_direction(-1)  # Set clockwise direction
#     ax[2,0] .set_theta_zero_location('N')  # Set zero angle at North

#     # Customize the tick labels on the circles (bins) so  that I can see the distance on the belt instead of degrees
#     circle_ticks = [0, 120, 240]  # Tick positions in degrees
#     circle_labels = ['0/150 cm', '50 cm', '100 cm']  # Labels for the ticks
#     ax[2,0].set_xticks(np.radians(circle_ticks))  # Set the tick positions in radians
#     ax[2,0].set_xticklabels(circle_labels)  # Set the tick labels

#     # Plot events on the circle
#     for i, events_round in enumerate(events_data):
#         event_angles = angles[np.where(events_round == 1)] #I found the angles where I have events
#         event_radii = np.ones_like(event_angles) * (i + 1)  # Adjust the radius for each round
#         ax[2,0].scatter(event_angles, event_radii, s=100, label=f'Round {i+1}', alpha=0.7)
#         for angle, radius in zip(event_angles, event_radii):
#             ax[2,0] .plot([0, angle], [0, radius], color='black', linestyle='-', linewidth=1)  # Connect center to event dot. better for visualization

#     # Calculate average direction using Cartesian coordinates
#     x_coords = []
#     y_coords = []
#     for i, events_round in enumerate(events_data):
#         event_angles = angles[np.where(events_round == 1)]
#         event_radii = np.ones_like(event_angles) * (i + 1)  # Adjust the radius for each round
#         x_coords.extend(np.cos(event_angles)/(i + 1))
#         y_coords.extend(np.sin(event_angles)/(i + 1))

#     # Sum the coordinates . I divide it with number of events. instead of number of rounds MAYBE THIS WOULD NEED TO CHANGE
#     sum_x = np.sum(x_coords)/ peak_n
#     sum_y = np.sum(y_coords)/ peak_n

#     # Convert the sum of coordinates to polar coordinates
#     sum_direction = np.arctan2(sum_y, sum_x)  # Note: y comes before x in arctan2
#     sum_magnitude = np.sqrt(sum_x**2 + sum_y**2)

#     # Plot the average tuning vector as an arrow
#     ax[2,0].arrow(0, 0, sum_direction, sum_magnitude, head_width=0, head_length=0,linewidth=4, fc='red', ec='red')

#     # Customize plot
#     ax[2,0].set_title(f'Events and Tuning Vectors Cell {cell}')
#     ax[2,0].legend(loc='upper left',bbox_to_anchor=(0.9, 1), frameon=False)
#     ax[2,0].grid(True)

#     # Convert average direction from radians to degrees
#     sum_direction_degrees = np.degrees(sum_direction)
#     # Map the sum direction to the corresponding position on the belt
#     position_on_belt = (sum_direction_degrees % 360) * (belt_length / 360)

#     ###########
#     #plotting the histogram of the vector lengths coming from the shuffled data and compare it with the actual vector length
#     ###########

#     shuffled_vector_lengths=[]

#     for n in range(shuffling_times):
#         #shuffling every row separately
#         shuffled_events_data = events_data.copy()  # Make a copy to avoid modifying original data
#         for row in shuffled_events_data:
#             np.random.shuffle(row)  # Shuffle the spiking events within each round independently

#         # Calculate average direction and magnitude for shuffled data. same logic as before exactly
#         shuffled_x_coords = []
#         shuffled_y_coords = []
#         for i, events_round in enumerate(shuffled_events_data):
#             event_angles = angles[np.where(events_round == 1)]
#             event_radii = np.ones_like(event_angles) * (i + 1)  # Adjust the radius for each round
#             shuffled_x_coords.extend(np.cos(event_angles)/(i + 1))
#             shuffled_y_coords.extend(np.sin(event_angles)/(i + 1))

#         shuffled_sum_x = np.sum(shuffled_x_coords)/ peak_n
#         shuffled_sum_y = np.sum(shuffled_y_coords)/ peak_n
#         shuffled_sum_direction = np.arctan2(shuffled_sum_y, shuffled_sum_x)
#         shuffled_sum_magnitude = np.sqrt(shuffled_sum_x**2 + shuffled_sum_y**2)
#         shuffled_vector_lengths.append(shuffled_sum_magnitude)

#         # Map the shuffled average direction to the corresponding position on the belt
#         shuffled_position_on_belt = (np.degrees(shuffled_sum_direction) % 360) * (150 / 360)


#     ###########
#     #making the histogram with the shuffled data and our actual vector length
#     ###########

#     # Calculate histogram
#     counts, bins = np.histogram(shuffled_vector_lengths, bins=20)

#     # Calculate probabilities
#     total_samples = len(shuffled_vector_lengths)
#     probabilities = counts / total_samples

#     # comparing it with the actual vector lenth now)
#     # Actual vector length
#     actual_length = sum_magnitude  # Example value


#     # Calculate the p-value
#     larger_lengths_count = np.sum(np.array(shuffled_vector_lengths) > actual_length)
#     total_samples = len(shuffled_vector_lengths)
#     p_value_vector = larger_lengths_count / total_samples

#     # Plot histogram with probabilities
#     ax[2,1].bar(bins[:-1], probabilities, width=np.diff(bins), color='blue', alpha=0.7)
#     ax[2,1].axvline(x=sum_magnitude,color='red')
#     ax[2,1].set_xlabel('Vector Length')
#     ax[2,1].set_ylabel('Probability')
#     # Remove right and upper axes
#     ax[2,1].spines['right'].set_visible(False)
#     ax[2,1].spines['top'].set_visible(False)
#     ax[2,1].set_title(f'Histogram for vector lengths\np-value:{p_value_vector:.4f}')
#     ax[2,1].grid(False)

#     ###################################################
#     #plotting the histogram from colmogorov smirnov test
#     ##################################################

#     ###########################
#     # I will shuffle once. and then I will compare the shufle data with my original data. the colmogorov smiron value will be the value that I will compare later
#     # later I will shuffle the data 1000 times and I will compare them to the first suffling data
#     ##############################

#     shuffled_ks=[] #array where I will put the ks distances where I will compare the shuffled data with my first shuffling
#     baseline=data.copy()

#     #shuffle 1 for the baseline
#     for i in range(num_rounds):
#         shuf=random.randint(1,150)
#         baseline[i]=np.roll(baseline[i],shuf)
#     baseline_avg=np.mean(baseline,axis=0)
    
#     baseline_ks,_=kstest(data_avg,baseline_avg)


#     # now I will shuffle many times and then compare
        
#     for n in range(1,shuffling_times):
#         data_shuffle=data.copy()    
#         for i in range(num_rounds):
#             shuf=random.randint(1,150)
#             data_shuffle[i]=np.roll(data_shuffle[i],shuf)
        
        

#         data_shuffle=np.mean(data_shuffle,axis=0)
#         ks_shuffle,p_value_=kstest(baseline_avg,data_shuffle)
#         shuffled_ks.append(ks_shuffle)

    
#     # Calculate histogram
#     counts, bins = np.histogram(shuffled_ks, bins=30)

#     # Calculate probabilities
#     total_samples = len(shuffled_ks)
#     probabilities = counts / total_samples
#     # Calculate the p-value
#     larger_lengths_count = np.sum(np.array(shuffled_ks) > baseline_ks)
#     p_value_ks = larger_lengths_count / total_samples
#     # Annotate the p-value on the upper side of the plot
#     # Create a legend entry with the p-value
#     p_value_text = f'P-value ks: {p_value_ks:.4f}'
#     #p_value_patch = mpatches.Patch(color='none', label=p_value_text)

#     # Add the legend with the custom entry
#     ax[3,1].bar(bins[:-1], probabilities, width=np.diff(bins), color='blue', alpha=0.7)
#     ax[3,1].axvline(x=baseline_ks,color='red')
#     ax[3,1].set_xlabel('ks Length')
#     ax[3,1].set_ylabel('Probability')
#     # Remove right and upper axes
#     ax[3,1].spines['right'].set_visible(False)
#     ax[3,1].spines['top'].set_visible(False)
#     ax[3,1].set_title(f'colmogorov smirnov test\np-value:{p_value_ks:.4f}')
#     ax[3,1].grid(False)

#     if p_value_vector < 0.05:
#         place_cells_tuned_vector.append(cell)
#     if p_value_ks < 0.05:
#         place_cells_ks_test.append(cell)

#     #ax[4,0]=None

#     #plotting cells morphology

#     cell_shape = np.reshape(spatial[:,cell], (512, 512)) # (262144 -> 512x512, i.e. "unflatten")
#     indices=np.where(cell_shape>0)

#     v_viz=40 #value for better visualization

#     #filtered=gaussian_filter(spatial,sigma=-1)

#     # Determine minimum and maximum row and column indices
#     min_row, max_row = np.min(indices[0]), np.max(indices[0])
#     min_col, max_col = np.min(indices[1]), np.max(indices[1])

#     # Extract the sub-array corresponding to the area where the cell appears
#     viz_values=np.array([min_row-v_viz,max_row+1+v_viz,min_col-v_viz,max_col+1+v_viz])
#     # Replace any value less than zero with zero
#     viz_values[viz_values < 0] = 0

#     sub_data = cell_shape[viz_values[0]:viz_values[1], viz_values[2]:viz_values[3]]

#     # Plot the sub-array using Seaborn's heatmap

#     sns.heatmap(sub_data, cmap='viridis',cbar=False,ax=ax[4,1])
#     ax[4,1].set_xticks([])
#     ax[4,1].set_yticks([])
#     ax[4,1].set_title(f'Cell Morphology')

#     ax[4,0].axis('off')

 


#     plt.subplots_adjust(hspace=0.8,wspace=0.2)
#     #plt.subplots_adjust(wspace=1)


#     #     # Check the p-value and save the figure to the appropriate PDF file
#     # if p_value_vector < 0.05 and p_value_ks < 0.05:
#     #     place_cells_both_methods.append(cell)
#     #     significant_cells.savefig(fig)
#     # else:
#     #     non_significant_cells.savefig(fig)
    
#     plt.ioff() #this is so I dont have to show the plots but only store them

#     #plt.close()

# # Close the PDF files after saving all figures
# # significant_cells.close()
# # non_significant_cells.close()


creating the plae code diagram for both methods colmogorov smiron and tuned vector analysis separatately
I do this with the zscore fluorescence and raw data at the same time

In [ ]:
avr_firing_rate_maps_df=pd.DataFrame(avr_firing_rate_maps) #converting the whole units array into a panda frame
avr_firing_rate_maps_raw_data_df=pd.DataFrame(avr_firing_rate_maps_raw_data) #converting the whole units array into a panda frame

place_cells_plotting=PdfPages(fig_folder+'/place_cell_coding_zscore.pdf')


cell_datasets=[place_cells_both_methods,place_cells_tuned_vector,place_cells_ks_test]
titles=['both methods','tuned vector','ks test']

custom_ticks = [0, 50, 100, 150]  # Specify the positions where you want the ticks
custom_labels = ['0', '50', '100', '150']  # Specify the labels for the ticks

fig2,ax=plt.subplots(1,3,figsize=(15,10))#
fig2.suptitle(f'{mouse_ID}\n{condition}\n Zscore fluorescence')


#########################################
# for zscore fluorescence
############################################

for i,(cell_dataset,ind_title) in enumerate(zip(cell_datasets,titles)):
    place_cell_activity=avr_firing_rate_maps_df.iloc[cell_dataset] #keeping only the place but without changing the indexing
    place_cell_indexes_max=np.argmax(place_cell_activity,axis=1) #finding the place of the max activity of every cell
    place_cell_indexed_filtered=place_cell_activity.apply(np.argmax, axis=1).sort_values().index #there are the new indexes of the place cells just so thez can form the nice place cell diagram
    original=place_cell_activity.index
    transpose=place_cell_indexed_filtered
    place_cell_activity=place_cell_activity.reindex(index=transpose) #reindexing the place cell panda frame according to the max activitz and the position in the belt
    with h5py.File(f'{save_folder}/output_info.h5', 'a') as file:
        # if f'sorted_place_cells_{ind_title}' in file:
        #     del file[f'sorted_place_cells_{ind_title}']  # Delete the existing dataset
        file.create_dataset(f'sorted_place_cells_{ind_title}', data=place_cell_activity)

    sns.heatmap(place_cell_activity,ax=ax[i],vmin=0,cmap='viridis',cbar_kws={'label': 'z-score fluorescence'})
    ax[i].set_xlabel('')
    ax[i].set_ylabel('Place cells')    
    ax[i].set_title(f'{ind_title}\n{len(cell_dataset)} place cells')
    ax[i].set_xticks(custom_ticks)
    ax[i].set_yticks([])
    ax[i].set_xticklabels(custom_labels)

# #########################################
# # for raw fluorescence
# ############################################

# for i,(cell_dataset,ind_title) in enumerate(zip(cell_datasets,titles)):
#     place_cell_activity=avr_firing_rate_maps_raw_data_df.iloc[cell_dataset] #keeping only the place but without changing the indexing
#     place_cell_indexes_max=np.argmax(place_cell_activity,axis=1) #finding the place of the max activity of every cell
#     place_cell_indexed_filtered=place_cell_activity.apply(np.argmax, axis=1).sort_values().index #there are the new indexes of the place cells just so thez can form the nice place cell diagram
#     original=place_cell_activity.index
#     transpose=place_cell_indexed_filtered
#     place_cell_activity=place_cell_activity.reindex(index=transpose) #reindexing the place cell panda frame according to the max activitz and the position in the belt

#     sns.heatmap(place_cell_activity,ax=ax[1,i],vmin=0,vmax=200,cmap='viridis',cbar_kws={'label': 'raw fluorescence'})
#     ax[1,i].set_xlabel('Distance in belt (cm)')
#     ax[1,i].set_ylabel('Place cells')    
#     ax[1,i].set_title('')
#     ax[1,i].set_xticks(custom_ticks)
#     ax[1,i].set_yticks([])
#     ax[1,i].set_xticklabels(custom_labels)



plt.show()

place_cells_plotting.savefig(fig2)

place_cells_plotting.close()